In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

import numiweight

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)

In [ ]:
plt.rcParams.update({'font.size': 16})
mpl.rcParams['lines.linewidth'] = 4.

dosave = True
savedir = "./plots_2_28_24/scalar-spectra/"

Masses = [220, 240, 280, 340]

In [ ]:
fhs = ["/icarus/data/users/gputnam/DMCP2023G/mc-F-iter2/F2-Higgs_M%i_nom_evt.df" % M for M in Masses]

In [ ]:
h_mchdfs = [pd.read_hdf(fh, key="mch") for fh in fhs]

In [ ]:
h_mcdfs = [pd.read_hdf(fh, key="mcnu") for fh in fhs]

In [ ]:
for i in range(len(h_mcdfs)):
    h_mcdfs[i]["when"] = SlcInFV(h_mcdfs[i].position)
    h_mcdfs[i]["M"] = h_mchdfs[i].M.iloc[0]
    
    h_mcdfs[i]["leadMuE"] = np.maximum(h_mcdfs[i].p0.genE, h_mcdfs[i].p1.genE)
    h_mcdfs[i]["sublMuE"] = np.minimum(h_mcdfs[i].p0.genE, h_mcdfs[i].p1.genE)

    h_mcdfs[i]["openangle"] = np.arccos(dotdf(h_mcdfs[i].p0.genp, h_mcdfs[i].p1.genp) / magdf(h_mcdfs[i].p0.genp) / magdf(h_mcdfs[i].p1.genp))*180/np.pi


In [ ]:
# get weights

for i in range(len(h_mcdfs)):
    nupdg = h_mcdfs[i].pdg.groupby(level=[0,1]).first()
    nuparent_pdg = h_mcdfs[i].parent_pdg.groupby(level=[0,1]).first()
    nuE = h_mcdfs[i].E.groupby(level=[0,1]).first()
    
    ppfx_cv = numiweight.cv(nupdg, nuE)
    concrete_cv = numiweight.concrete_cv(nuparent_pdg, nuE)
    
    h_mcdfs[i]["weight"] = ppfx_cv * concrete_cv
    h_mchdfs[i]["weight"] = ppfx_cv * concrete_cv

In [ ]:

thetaS = h_mchdfs[0].C1.iloc[0]
thetaS_STR = "1 \\times 10^{-5}" 
thetaS, thetaS_STR

In [ ]:
bins = np.linspace(0, 5, 21)

for df in h_mcdfs:
    plt.hist(df.E[df.when], bins=bins, density=True, weights=df.weight[df.when],
             histtype="step", label="$M_S = %i$ MeV" % (df.M.iloc[0]*1e3), linewidth=2)

plt.ylabel("Fiducial Events, Area Normalized")
plt.legend(title="Scalar Decays\n$\\theta_S = %s$" % thetaS_STR)

plt.xlabel("Scalar Energy [GeV]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "scalar_energy.pdf")

In [ ]:
bins = np.linspace(0, 5, 21)

for df in h_mcdfs:
    plt.hist(df.leadMuE[df.when], bins=bins, density=True, weights=df.weight[df.when],
             histtype="step", label="$M_S = %i$ MeV" % (df.M.iloc[0]*1e3), linewidth=2)

plt.ylabel("Fiducial Events, Area Normalized")
plt.legend(title="Scalar Decays\n$\\theta_S = %s$" % thetaS_STR)

plt.xlabel("Leading Muon Energy [GeV]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "leading_muon_energy.pdf")

In [ ]:
bins = np.linspace(0, 3, 13)

for df in h_mcdfs:
    plt.hist(df.sublMuE[df.when], bins=bins, density=True, weights=df.weight[df.when],
             histtype="step", label="$M_S = %i$ MeV" % (df.M.iloc[0]*1e3), linewidth=2)

plt.ylabel("Fiducial Events, Area Normalized")
plt.legend(title="Scalar Decays\n$\\theta_S = %s$" % thetaS_STR)

plt.xlabel("Sub-Leading Muon Energy [GeV]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "subleading_muon_energy.pdf")

In [ ]:
bins = np.linspace(0, 140, 15)

for df in h_mcdfs:
    plt.hist(df.openangle[df.when], bins=bins, density=True, weights=df.weight[df.when],
             histtype="step", label="$M_S = %i$ MeV" % (df.M.iloc[0]*1e3), linewidth=2)

plt.ylabel("Fiducial Events, Area Normalized")
plt.legend(title="Scalar Decays\n$\\theta_S = %s$" % thetaS_STR)

plt.xlabel("Muon Opening Angle [$^\\circ$]")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "openangle.pdf")

In [ ]:
bins = np.linspace(0.5, 1, 21)

for df in h_mcdfs:
    plt.hist(df.leadMuE[df.when] / (df.leadMuE[df.when] + df.sublMuE[df.when]), bins=bins, density=True, 
             weights=df.weight[df.when],
             histtype="step", label="$M_S = %i$ MeV" % (df.M.iloc[0]*1e3), linewidth=2)

plt.ylabel("Fiducial Events, Area Normalized")
plt.legend(title="Scalar Decays\n$\\theta_S = %s$" % thetaS_STR)

plt.xlabel("Leading Muon Energy Fraction")

plt.tight_layout()
if dosave:
    plt.savefig(savedir + "leading_muon_energy_frac.pdf")